In [28]:
import torch
from tab_transformer_pytorch import FTTransformer
import torch.nn as nn


In [29]:


model = FTTransformer(
    categories = [65536, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2], 
    num_continuous = 65,
    dim = 32,
    dim_out = 3,
    depth = 6, 
    heads = 8,
    attn_dropout = 0.1,
    ff_dropout = 0.1, 
)
print(model)
#model = model.load_state_dict(torch.load('model/Model_CICIDS2017_Task_A.pth'))

FTTransformer(
  (categorical_embeds): Embedding(65558, 32)
  (numerical_embedder): NumericalEmbedder()
  (transformer): Transformer(
    (expand_streams): Reduce('b ... -> (b s) ...', 'repeat', s=4)
    (reduce_streams): Reduce('(b s) ... -> b ...', 'sum', s=4)
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): HyperConnections(
          (branch): Attention(
            (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
            (to_qkv): Linear(in_features=32, out_features=384, bias=False)
            (to_out): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (act): Tanh()
          (split_fracs): Rearrange('b ... (f d) -> b ... f d', f=1)
          (merge_fracs): Rearrange('b ... f d -> b ... (f d)')
          (norm): RMSNorm()
          (dropout): Dropout(p=0.0, inplace=False)
          (residual_transform): Identity()
        )
        (1): HyperConnections(
          (bra

In [31]:
print(model.to_logits[-1])

Linear(in_features=32, out_features=3, bias=True)


In [27]:
# Incremental learning - Keep the old weights from original class add new classes neuron

old_head = model.mlp.mlp[-1]  # Linear(962, 3) 
 
in_features = old_head.in_features
new_head = torch.nn.Linear(in_features, 8)  # 8 = total classes for task A and B

with torch.no_grad():
    # copy Task A weights
    new_head.weight[:3].copy_(old_head.weight)
    new_head.bias[:3].copy_(old_head.bias)

# replace last layer
model.mlp.mlp[-1] = new_head.to(device)


torch.Size([3, 962])


In [24]:
print(newhead)

Linear(in_features=962, out_features=8, bias=True)
